## 1. Relevant imports



In [16]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn import preprocessing

## 2. Getting the data

In [17]:
vix = yf.Ticker("^vix") #instanciate vix data

historical_price_data = vix.history(period="max") # getting all historical data, this will return a dataframe object

#### Let's have a look at the raw data that we found

In [18]:
print(historical_price_data)

                 Open       High        Low      Close  Volume  Dividends  \
Date                                                                        
1990-01-02  17.240000  17.240000  17.240000  17.240000       0          0   
1990-01-03  18.190001  18.190001  18.190001  18.190001       0          0   
1990-01-04  19.219999  19.219999  19.219999  19.219999       0          0   
1990-01-05  20.110001  20.110001  20.110001  20.110001       0          0   
1990-01-08  20.260000  20.260000  20.260000  20.260000       0          0   
...               ...        ...        ...        ...     ...        ...   
2021-11-23  20.240000  20.910000  19.030001  19.379999       0          0   
2021-11-24  19.170000  20.959999  18.520000  18.580000       0          0   
2021-11-26  26.620001  28.990000  23.879999  28.620001       0          0   
2021-11-29  25.309999  25.690001  21.709999  22.959999       0          0   
2021-11-30  26.230000  28.559999  23.709999  27.190001       0          0   

## 3. Deal with missing data
- we see that the "Volume", "Dividends" and "Stock Splits" columns are empty. Why?
- Either fill in missing data if possible or just don't use it

#### We proceed to discard the empty columns

In [19]:
historical_price_data_cleaned = historical_price_data.drop(["Volume", "Dividends", "Stock Splits"], axis = 1)

#### OR

In [20]:
# historical_price_data.drop(["Volume", "Dividends", "Stock Splits"], axis = 1, inplace = True)

#### Lets check out our data now

In [21]:
print(historical_price_data_cleaned)

                 Open       High        Low      Close
Date                                                  
1990-01-02  17.240000  17.240000  17.240000  17.240000
1990-01-03  18.190001  18.190001  18.190001  18.190001
1990-01-04  19.219999  19.219999  19.219999  19.219999
1990-01-05  20.110001  20.110001  20.110001  20.110001
1990-01-08  20.260000  20.260000  20.260000  20.260000
...               ...        ...        ...        ...
2021-11-23  20.240000  20.910000  19.030001  19.379999
2021-11-24  19.170000  20.959999  18.520000  18.580000
2021-11-26  26.620001  28.990000  23.879999  28.620001
2021-11-29  25.309999  25.690001  21.709999  22.959999
2021-11-30  26.230000  28.559999  23.709999  27.190001

[8042 rows x 4 columns]


## 4. Nominal Values? Depends. In most cases, no
#### Lets now change the nominal values to a percentage change value between each day (daily is not a requirement, we could also have a rolling average of the last $x$ trading sessions)

In [22]:
historical_price_data_cleaned_pct_change = historical_price_data_cleaned.pct_change()

historical_price_data_cleaned_pct_change = historical_price_data_cleaned_pct_change[1:] # chops off the first NaN value

#### Lets look at out data again

In [23]:
print(historical_price_data_cleaned_pct_change)

                Open      High       Low     Close
Date                                              
1990-01-03  0.055104  0.055104  0.055104  0.055104
1990-01-04  0.056624  0.056624  0.056624  0.056624
1990-01-05  0.046306  0.046306  0.046306  0.046306
1990-01-08  0.007459  0.007459  0.007459  0.007459
1990-01-09  0.095755  0.095755  0.095755  0.095755
...              ...       ...       ...       ...
2021-11-23  0.112088  0.067381  0.096830  0.010955
2021-11-24 -0.052866  0.002391 -0.026800 -0.041280
2021-11-26  0.388628  0.383111  0.289417  0.540366
2021-11-29 -0.049211 -0.113832 -0.090871 -0.197764
2021-11-30  0.036349  0.111717  0.092123  0.184234

[8041 rows x 4 columns]


#### Lets take a peek at the summary statistics of our data

In [24]:
historical_price_data_cleaned_pct_change.describe()

,Open,High,Low,Close
count,8041.000000,8041.000000,8041.000000,8041.000000
mean,0.002527,0.002436,0.001848,0.002370
std,0.072105,0.071073,0.061013,0.070248
min,-0.358284,-0.370974,-0.400595,-0.295727
25%,-0.039051,-0.036530,-0.033166,-0.036927
50%,-0.002738,-0.002682,-0.003086,-0.003623
75%,0.036082,0.033333,0.032042,0.032787
max,1.023861,1.172452,0.601269,1.155979


## 5. Make Input data small values. They work better. i.e. 0 - 1 or -1 - 1
- standardize data
- normalize data

#### Iterate over the columns of data in our dataframe and normalise the data

In [25]:
for column in historical_price_data_cleaned_pct_change:

    mean = np.mean(historical_price_data_cleaned_pct_change[column])
    std = np.std(historical_price_data_cleaned_pct_change[column])
    historical_price_data_cleaned_pct_change[column] = (historical_price_data_cleaned_pct_change[column] - mean)/std # vectorised operation to standardise all columns

In [26]:
historical_price_data_cleaned_pct_change

,Open,High,Low,Close
Date,,,,
1990-01-03,0.729224,0.741096,0.872918,0.750738
1990-01-04,0.750306,0.762483,0.897832,0.772377
1990-01-05,0.607194,0.617294,0.728704,0.625482
1990-01-08,0.068403,0.070682,0.091967,0.072452
1990-01-09,1.293032,1.313088,1.539220,1.329446
...,...,...,...,...
2021-11-23,1.519559,0.913842,1.556836,0.122216
2021-11-24,-0.768272,-0.000627,-0.469565,-0.621396
2021-11-26,5.355049,5.356434,4.713503,7.658970


In [27]:
historical_price_data_cleaned_pct_change.describe()

,Open,High,Low,Close
count,8.041000e+03,8.041000e+03,8.041000e+03,8.041000e+03
mean,-1.060380e-17,1.413840e-17,-1.546387e-17,-5.301898e-18
std,1.000062e+00,1.000062e+00,1.000062e+00,1.000062e+00
min,-5.004291e+00,-5.254207e+00,-6.596403e+00,-4.243726e+00
25%,-5.766636e-01,-5.482770e-01,-5.739195e-01,-5.594304e-01
50%,-7.302347e-02,-7.201580e-02,-8.088144e-02,-8.531467e-02
75%,4.653978e-01,4.347568e-01,4.949057e-01,4.330235e-01
max,1.416545e+01,1.646317e+01,9.825039e+00,1.642290e+01


#### We see that the extreme values are still really big. This may cause trouble in training We can now normalize our data so it is between 0 and 1

In [28]:
scaler = preprocessing.MinMaxScaler() #Import the min max scalar tool

historical_price_data_cleaned_pct_change_copy = historical_price_data_cleaned_pct_change # I made a copy for testing purposes. If I messed up I could go back

# Pick the columns that we want to normalise. In our case, all.
historical_price_data_cleaned_pct_change_copy[["Open", "High", "Low", "Close"]] = scaler.fit_transform(historical_price_data_cleaned_pct_change_copy[["Open", "High", "Low", "Close"]])

#### Lets check out our data again

In [29]:
historical_price_data_cleaned_pct_change_copy

,Open,High,Low,Close
Date,,,,
1990-01-03,0.299092,0.276060,0.454852,0.241668
1990-01-04,0.300192,0.277045,0.456369,0.242715
1990-01-05,0.292726,0.270360,0.446070,0.235607
1990-01-08,0.264620,0.245190,0.407295,0.208848
1990-01-09,0.328503,0.302398,0.495427,0.269670
...,...,...,...,...
2021-11-23,0.340320,0.284014,0.496500,0.211256
2021-11-24,0.220974,0.241907,0.373100,0.175274
2021-11-26,0.540401,0.488578,0.688728,0.575938


In [30]:
historical_price_data_cleaned_pct_change_copy.describe()

historical_price_data_cleaned_pct_change_copy.to_csv("/Users/andrewchen/Documents/GitHub/adrinet/data/cleaned/vix_cleaned_andrew.csv")

# DONE